In [1]:
from config import setup
setup()

 # Projet AP : Late Evacuation Planning Problem
 
Team : ADA
 
Team members :
- Anaïs Rabary  
- Duc Hau Nguyen  
- Adrien Mega  

Plan :
1. Introduction au projet GEO-SAFE
2. Evacuation Planning problem
3. Generator
4. 1ers Résultats  avec la contrainte cumulative
5. Conclusion




## Partie 1 :  Introduction projet GEO-SAFE

Le but de ce projet est de pouvoir **assister la prise de décision des autorités lors de l'évacuation à grande échelle (de la population) pendant un feu de forêts**. Pour cela, il faut modéliser le problème pour vérifier ensuite que  l'évacuation de la population pendant un feu est bien réalisable. 

La population se trouve sur des noeuds à évacuer. Dans la "vraie vie", il y a 3 catégories de personnes à évacuer : les personnes qui partent tôt, celles qui se réfugient dans un lieu et celles qui restent "se battre". Les derniers sont les personnes qui concernent l'objet de la modélisation. Ce sont des personnes qui partent tard.

Ce problème d'évacuation est souvent comparé à l'évacuation lors d'innondations. Mais dans ce dernier cas, les modèles sont plus "simples" car l'eau évolue toujours de la même façon. Ce n'est pas le cas du feu qui est beaucoup moins prédictible que l'eau. En effet, la propagation d'un feu n'est pas fixe pour une topologie donnée mais évolue en fonction du vent, de la présence de combustibles, ... Donc la méthode d'évacutaion lors de feu a besoin d'être plus robuste pour pouvoir prédire différents scénarios. 

En plus de minimiser le temps d'évacuation de la population, le but est de maximiser le minimum "safety margin" spatial et temporel pondéré par la population, sur chaque segment de route. Il faut donc être le plus loin et le plus rapidement possible du feu.

L'équipe ADA s'est appuyée sur le papier *A study of Evacuation Planning for Wildfires* de l'équipe GeoSafe (Christian Artigues, Emmanuel Hebread, Yannick Pencole, Andreas Schutt et Peter Stuckey). La méthode d'optimisation choisie ici est PCC, **programmation par contraintes** avec CP Optimizer.

Dans un premier temps, nous présenterons la **modélisation du problème**. Nous intronduirons aussi le **module de génération de données et son exploitation** pour notre problème. Puis nous présenterons notre **résolution pour un simple modèle à l'aide d'une contrainte cummulative**. 

## Partie 2 : Evacuation Planning Problem

Il faut déterminer la date à laquelle l'évacuation commence pour chaque noeud, ainsi que le taux d'évacuation. Le taux d'évacuation est en fait une modélisation du niveau de ressources utilisées par les autorités pour évacuer les gens.  

### 2.1 Hypothèses
Quelques hypothèses ont été posées par l'équipe GeoSafe :
- Les authorités ont déjà identifié des routes accessibles et des points de refuges en sécurité. Chaque sommet à évacuer a donc une seule route d'évacuation. Il y a 1 seul et même point de refuge à atteindre par toute la population a évacuer. Il se peut qu'il y ait une congestion sur un des segment de route partagé par plusieurs points.
- Les autorités ont déja estimé le nombre d'habitants à évacuer (la population de ceux qui partent tard).
- La modélisation du feu donne une deadline pour chaque segment (date à laquelle le feu arrive).
- Une fois que l'évacuation d'un point a commencé, le process ne peut pas être interrompu. Les personnes en cours d'évacuation ne s'arrêtent jamais.
- C'est non-préemptif.
- Le flux d'évacuation reste constant durant tout le processus d'évacuation.

### 2.2 Modélisation 
Un arbre $G= (\varepsilon \cup T \cup \{r\}, A)$ qui représente les routes depuis les sommets à évacuer $\varepsilon$ vers le refuge $r$ en passant par des noeuds de transit $T$

- sur chaque noeud $v\in\varepsilon$, il y a $W_{v}$ personnes à évacuer.
- Chaque arc alant de $u$ à $u'$, de longueur $l_{uu'}$ a une capacité $q_{u}$.
- $H=[0,h]$ représente l'interval de temps pour l'évacuation.
Il faut associer chaque $v$ à un réel $s_{v}$ représentant le **retard du préavis d'évacuation** ainsi qu'a une courbe de réponse $\phi_{v}$ décrivant le **flow d'évacuation** d'un noeud $v$ a un taux $\phi_{v}(t)$. Avant $s_{v}$ le taux $\phi_{v}$ est à 0.
$$\int_{0}^{H}\phi_{v}(t)dt=w_{v} $$
Le flow $\phi_u$ pour n'importe quel arc $u-v$ à n'importe quel moment noeud : 
$$\phi_u(t)=\sum_{v\in descendants(u)} \phi_{v}(t-l_{uv}-s_{v})$$  

On peut aussi le réécrire :
$$\phi_u(t)=\sum_{V\in descendants(u),\: s_v+l_{uv}\le t \lt s_v+l_{uv}+\frac{w_v}{h_v}} h_v$$  

Ici, $\phi_v$ est une courbe de réponse simple. Le flux sortant $\phi_v$ est une variable de décision continue et qui reste constante penant le process d'évacuation.
On note aussi :
$$ \phi_v = h_v\: during\: [s_v,e_v]\: with\: e_v=s_v + \frac{w_v}{h_v}$$


- Il y a une tâche pour chaque noeud à évacuer. Pour chaque tâche, on a un **taux constant d'évacuation** $\phi_v \le q_{v}$ et une **heure de début d'évacuation** $s_{v}\in [0, H-\frac{w_v}{h_v}]$

**CONTRAINTES** :   
Il n'y a qu'un seul type de contraintes pour éviter les bouchons (donc éviter d'avoir un flux plus grand que la capacité d'un arc) : $\phi_v \le q_{v}$

**OBJECTIF** :  
Chaque noeud de transit a une date au plus tard (due date) $d_u$, date à partir de laquelle la route n'est plus en sécurité (à cause du feu). L'objectif est donc de minimiser l'écart de temps maximal entre lequel la population quitte $u$ et la due date $d_u$.
$$\min \max_{u\in T,v\in descendants(u)} s_v + \frac{h_v}{w_w} + l_{uv} - d_u$$

Cette fonction objectif se simplifie en 
$$\min \max_{v\in\varepsilon} s_v + \frac{h_v}{w_w} - d_v$$
avec $d_v=\min_{u\in T}\{d_u-l_{uv}\}$

**OBSERVATION**:   
Pour chaque noeud de transit $u$ et $u'$, avec $u'$ déscendant de $u$ un bouchon sur $u'$ entraine un bouchon sur $u$. Donc, il ne faut checker l'occurance de bouchons uniquement si la capacité de $u$ est supérieur à la capacité de sons ascendant $v$ ( $\forall v \in p'(u),q_u \gt q_v$ )  

**SIMPLIFICATION**:     
Sur un chemin de $u$ vers $u'$ sans branches, on garde la capacité de l'arc minimum. 
Par exemple, avec c'<c, on garde donc la capacité c' :  
            S5  `  
```
S1 ___ S2 ___ S3 ___ S4  
     c     c'  |    
               S5  
```
   
```
S1 ___ S2 ___ S3 ___ S4  
     c'     c'  |    
               S5  
```

Ci-après le code de cette modélisation

In [ ]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np
import math

In [ ]:
from config_duc import setup
setup()

In [ ]:
def geosafe_model(l, q, eps, r, W, H, d):
    '''       
    Int  Matrix l: length path from i to j = G.l[i,j]. No path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    Int  Array  W: initial population at node i <--> W[i]
    Int         H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0]
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    s = np.array( mdl.integer_var_list(nb_node, min=0, max = H, name="s") )
        
    # evacuation rate aka. height of package
#     h = np.array([
#         mdl.integer_var(name='phi[%d]')%(i+1), min=0, max = W[i] for i in range(nb_nodes)
#     ])
    h = np.array( mdl.integer_var_list(nb_node, min=0, max = 100, name="h") )

    # == Intermediate ==
    # node flow of population
    phi = np.matrix([
            [ 
                mdl.integer_var(name='phi[%d,%d]'%(i+1,j)) for j in range(H) 
            ] for i in range(nb_node)
        ])
    
    # ending date (leaving time) of node
    e = np.array( mdl.integer_var_list(nb_node, min=0, max = 150, name= "e") )
    
    for v in range(nb_node):
        if eps[v]:
            mdl.add(
                e[v] == s[v] + W[v]/h[v]
            )
    
    # == Constraints ==
    
    # Time long enough to evacuate (additional)
    for v in nodes[eps]:
        mdl.add( h[v] * H >= W[v] )
    
    # evacuate everyone in evacuation node (???)
    # has been moved above
    
    # Flow at a node u = sum from all of its leaves node epsilon
#     mdl.add(
#         phi[u, t] == np.sum( phi[u, t - l[u,v] - s[eps]] ) \
#             for t in range(H)                                  \
#             for v in np.where(eps)                           \
#             for u in np.where(t)                             
#     )

    # simplified version
#     for u in nodes[np.where(np.logical_and(np.invert(eps), np.invert(r)))]:
        
#         for t in range(H):
            
#             v_arrays = nodes[np.where(
#                                 np.logical_and(
#                                     np.logical_and(
#                                         np.logical_and(
#                                             eps,
#                                             l[:,u] > 0, 
#                                         ),
#                                         s + l[:,u] <= t,
#                                     ),
#                                     t <= e + l[:, u]
#                                 )
#                             )[1]]
            
#             mdl.add(
#                 phi[u, t] == mdl.sum(h[v] for v in v_arrays)
#             )

#     for u in nodes[eps]:
#         mdl.add(
#             mdl.sum(phi[u, t] for t in range(H)) == h[u]
#         )

    for t in range(H):

        # Flow at epsilon should be h during the evacutation period
        for u in nodes[eps]:
            mdl.add( 
                phi[u, t] == h[u]
            )
            
        
        # Flow at a node u = sum from all of its leaves node epsilon
        for u in nodes[np.where(np.logical_and(np.invert(eps), np.invert(r)))]:
            mdl.add(
                phi[u, t] == mdl.sum(h[v] for v in nodes[eps] if l[v,u] != math.inf and s[v] + l[v,u] <= t and t <= e[v] + l[v,u])
            )
    
    # Flow at a node does not excess capacity of arc
    for t in range(H):
        for u in nodes:
            mdl.add(
                phi[u,t] <= q[u]
            )
            
    # Objective
#     mdl.add(
#         minimize(mdl.max(s[eps] + h[eps]/W[eps] - d[eps]))
#     )
    mdl.add(
        minimize(mdl.max(s[u] + W[u]/h[u] - d[u] for u in nodes[eps] ))
    )
    
    return mdl


### 2.3 Utilisation des contraintes cumulatives
Une première approche se fait avec des contraintes cumulatives simples.     

**NOTATION** :   
Pour une variable $x$, $\bar{x}$ signifie la plus grande valeur de x et $\underline{x}$, la plus petite valeur de x. 

Si on prend une selection de **tâches** $J$ avec une **heure de départ** $s_{i} \in [\underline{s_i}, \bar{s_i}]$, avec une **durée de process** $p_i \in [\underline{p_i},\bar{p_i}]$, une **hauteur** $h_i \in [\underline{h_i}, \bar{h_i}]$ et une **resource** $r$ avec une **capacité constante** $q_r$, on défini la contrainte cumulative comme :
$$ \sum_{i\in J|s_i\le t\le s_i+p_i} h_i \le q_r, \forall t\in H$$

Donc pour modéliser le problème, il suffit d'associer une tâche $v$ à chaque noeud à évacuer ($v \in \varepsilon$), avec une hauteur $h_v \in ]0, q_v]$, avec une heure de départ $ s_v \in [0, H-\frac{w_v}{q_v}]$, une heure de "complétion" $e_v \in [\frac{w_v}{q_v}, H]$. Puis il faut dupliquer et transformer cette tâche pour chaque arc critique de transit jusqu'au point de refuge. On note $i_{uv}$ la duplication du noeud $v$ sur l'arc critique $u$.

On a donc :
$$cumulative((s_{i_{uv}}, e_{i_{uv}} - s_{i_{uv}}, h_{i_{uv}})_{v\in L(u)}, q_u) \forall u \in T $$
$$w_v=h_v(e_v-s_v) \forall v \in \varepsilon$$
$$s_{i_{uv}} = s_v +l_{uv} \forall u\in T, \forall v \in L(u)$$
$$e_{i_{uv}} = e_v +l_{uv} \forall u\in T, \forall v \in L(u)$$
$$h_{i_{uv}} = h_v \forall u\in T, \forall v \in L(u)$$

Le problème avec les contraintes cumulatives "classiques" c'est qu'elles ne considèrent que les bornes inférieurs des domaines de solutions possibles. (*cf.exemple p7 de l'article*). Elles ne permettent pas de raisonner avec l'énergie totale de chaque tâche.

Des évolutions ont été proposées par l'équipe GeoSafe mais n'ont pas été étudiées ici.


------------------------------------------------------
**NOTE** : Le code associé à cette modélisation se trouve en partie 4 où sont détaillés les 1ers résultats

-------------------------------------------------------

## Partie 3 :  Generator 
Generator est une instance qui permet de générer aléatoirement un réseau de routes et de propager un simple model de feu. Les données générées prennent en compte des propagations de contraintes qu'il convient d'expliquer et de comprendre.
 
### 3.1 Install module
 Pour générer les données utilisées ensuite dans le projet, il faut utiliser le générateur du projet evacsim présent sur le git https://github.com/ehebrard/evacsim
 Il faut l'utiliser avec python 2 et avoir installé decorator et networkx.
 Note : il faut un dossier data dans le projet. C'est dans ce dossier que seront déposées les données générées.  
 
 ***
 
 Commande pour installer :  
 `python setup.py install --user`
 
 Les commandes pour générer :  
 `python generator.py --road test --printroad`  
 `python generator.py test --evacuation --printfire --seed 10`
 
 NOTE : avec la commande : `python generator.py --road test --evacuation --printfire --seed 10 --printevac`, on peut voir la route d'évacuation  
 
 ***  
 
 Pour générer une nouvelle instance de feu, il suffit de changer le paramètre seed.
 
 ### 3.2 Modélisation données générées
 FORMAT :
 n m population_1 maximum_rate_1 duedate_1 ... population_n maximumrate_n duedate_n capacity_1 k_1 i_1_1 offset_i_1_1 ... i_k_1 offset_i_k_1 ... capacity_m k_m i_1_m offset_i_1_m ... i_k_m offset_i_k_m
 
- **n**, le nombre de noeuds à évacuer
- **m**, le nombre d'arcs de transit pertinents (les arcs critiques)
- **Population_i**, population du noeud i a évacuer.
- **Maximum_rate_i**, taux max de personnes pouvant être évacuées en même temps
- **duedate_i**, Date à laquelle le noeud doit être évacué
- **capacity_y**, la capacité de l'arc de transit y
- **k_y**, le nombre de groupes de population transitant par cet arc
- **offset_i_x_y**, date à laquelle la population du groupe i_x_y atteind cet arc si elle part à t=0

EXEMPLE :   
- l1 : `10 8`  
10 zones à évacuer avec 8 arcs à utiliser pour évacuer les zones  
- l2 : `1677 70 66`   
sur le sommet 1, 1677 personnes à évacuer, par pacquets de 70, avant la date 66  
- l3 : `4161 70 40`  
- l4 : `3817 70 36`  
- l5 : `3745 70 92`  
- l6 : `1379 72 126`   
- l7 : `3359 71 115`  
- l8 : `893 72 120`  
- l9 : `463 72 54`  
- l10: `4368 212 86`  
- l11: `4987 70 44`  
Sommet 10  
- l12: `74 2 3 20 6 8`  
Arc 1, capacité de 74 personnes, Nombre de Groupes transitant par cet arc(2), puis lire par tuple de 2 (3,20) : 3, num du groupe concerné, 20, date au plus tôt à laquelle la population peut partir si il part à t=0. Cela veut donc dire que si le groupe 3 part a t=4, il pourra traverser l'arc 1 à partir de t=24.(6,8).  
- l13: `74 4 2 1 5 2 7 4 9 2`  
arc2, capacité74 personnes, 4 groupes possibles puis (2,1), (5,2), (7,4), (9,2)  
- l13: `206 8 0 13 1 15 2 16 4 14 5 17 7 19 8 9 9 17`  
- l14: `72 2 0 2 1 4`  
- l15: `203 10 0 33 1 35 2 36 3 40 4 34 5 37 6 28 7 39 8 29 9 37`  
- l16: `72 5 2 4 4 2 5 5 7 7 9 5`  
- l17: `72 2 7 3 9 1`  
- l18: `138 7 0 8 1 10 2 11 4 9 5 12 7 14 9 12`  
Arc 8 capacité de 138 personnes, ...    

Ci-après 2 images représentant les données des routes, du feu et des chemins d'évacuation générées.  
**NOTE IMPORTANTE** : les couleurs des arcs correspondent à la capacité de la route.

<div class="row" style="margin-top: 10px">
    <div class="col-md-offset-0 col-md-3">
        <img src="evacsim-master/image_Routes_Anais/routeFire.png" style="margin-right: 0; width: 400px;" />
    </div>  
    <div class="col-md-offset-0 col-md-3">
        <img src="evacsim-master/image_Routes_Anais/routeEvacuation.png" style="margin-right: 0; width: 300px;" />
    </div>
</div>

### 3.3 Récupération des données générées

In [1]:
import math
import json
import networkx as nx
from docplex.cp.model import CpoModel
from docplex.cp.model import *

# Comme on ne peut pas définir la longueur d'un arc à partir des fichiers générés ...
VALEUR_ARBITRAIRE_ARC_SORTIE = 1

Une première fonction parse le fichier graphe écrit par le générateur pour en récupérer le contenu.
Ces données sont simplement retournées sous forme de listes (une pour les zones, une pour les arcs).

In [2]:
def read_evac(filename):
    
    with open(filename,"r") as file:
        content = file.readlines()
    
    if len(content) > 0:
        
        nbzones, nbarcs = content[0].split(" ")
        content.remove(content[0])
    
        lineszones = content[0:int(nbzones)]
        linesarcs  = content[int(nbzones):int(nbzones)+int(nbarcs)]
        
        E = []
        
        for line in lineszones:
            personnes, paquets, datemax = line.split(" ")
            E.append([int(personnes),int(paquets),int(datemax)])
            
        A = []
        
        for line in linesarcs:
            
            content = line.split(" ")
            capacite = content[0]
            content = content[2:]
            
            groups = []
            
            for i in range(0,len(content),2):
                numgroupe = content[i]
                datemax = content[i+1]
                
                groups.append([int(numgroupe),int(datemax)])
                
            A.append([int(capacite),groups])
            
        return E,A
        
        
# import os
# os.listdir()

E,A = read_evac("evacsim-master/data/test_10_25_2_10.evac")

print(*E, sep = "\n")
print(*A, sep = "\n")

[4455, 70, 68]
[509, 70, 22]
[4584, 71, 128]
[2902, 71, 122]
[2808, 71, 84]
[2744, 70, 22]
[3982, 3982, 50]
[1471, 70, 122]
[1783, 71, 22]
[358, 70, 68]
[71, [[1, 0], [8, 0]]]
[224, [[2, 17], [4, 35], [5, 30]]]
[70, [[1, 3], [6, 3], [8, 3]]]
[132, [[4, 29], [5, 24]]]
[132, [[0, 0], [3, 8], [9, 0]]]
[249, [[0, 18], [1, 23], [2, 30], [3, 26], [4, 48], [5, 43], [6, 23], [8, 23], [9, 18]]]
[132, [[3, 8], [9, 0]]]
[146, [[0, 1], [1, 6], [3, 9], [6, 6], [8, 6], [9, 1]]]


Une lecture et un réarrangement sont ensuite appliqués afin de déduire et regrouper les données qui seront utiles à notre modèle de résolution. Ces données sont restituées sous la forme d'une liste plus complexe.

Après avoir lu les propriétés de chaque sommet à évacuer, les arcs sont parcourus et ajoutés à la liste des arcs d'un sommet à évacuer. Les arcs sont ajoutés dans l'ordre de parcours.

La dernière étape de cette fonction consiste à identifier les sommets intermédiaires entre les arcs, ainsi que le(s) sommet(s) de sûreté. Cette étape nous permet d'obtenir plusieurs propriétés des arcs que nous n'avions pas jusque là, comme le sommet d'origine / d'arrivée ou le temps de parcours d'un arc.

In [3]:
def graph2list(filename):
    
    E,A = read_evac("evacsim-master/data/test_10_25_2_10.evac")
    
    ########################## TRAITEMENT DES SOMMETS ##########################
    
    new_E = [] # Ensemble de sommets à évacuer
    
    for i_k in range(len(E)):
        w_k, h_k, d_k = E[i_k]
        p_k = math.ceil(w_k / h_k)
        A_k = []
        q_k = None
        
        k = [
            i_k,  # Identifiant du sommet
            w_k,  # Nombre de personnes à évacuer
            h_k,  # Taux d'évacuation d'un sommet
            d_k,  # Date à laquelle le sommet crame
            p_k,  # Durée d'évacuation d'un sommet
            A_k,  # Liste des sommets pour le chemin d'évacuation
            q_k,  # Capacité d'un sommet
        ]
        
        new_E.append(k)
        
    ########################## TRAITEMENT DES ARCS ##########################
    
    new_A = []
    
    for i_e in range (len(A)):
        c_e, groups = A[i_e]
        in_e = None
        out_e = None
        l_e = None

        for i_k, b_e in groups:
            
            e = [    # Arc d'un chemin d'évacuation d'un sommet
                i_e, # Identifiant de l'arc
                b_e, # Date min de passage
            ]
            
            k = new_E[i_k]
            A_k = k[5]
            
            if len(A_k) == 0:
                A_k.append(e)
            else:
                added = False
                for i in range (len(A_k)):
                    if A_k[i][1] > e[1]:
                        A_k.insert(i,e)
                        added = True
                if added == False:
                    A_k.append(e)
            
            k[5] = A_k
            new_E[i_k] = k

        e = [
            i_e,     # Identifiant de l'arc
            c_e,     # Capacité de l'arc en personnes par unité de temps
            in_e,    # Identifiant du sommet entrant
            out_e,   # Identifiant du sommet sortant
            l_e,     # Longueur de l'arc
        ]
        
        new_A.append(e)
        
    ####################### IDENIFICATION DES SOMMETS #######################
    
    # Cette partie va servir à trouver et compléter la liste des sommets
    # intermédiaires et de sortie, tout en complétant les in/out des arcs
        
    new_T = [] # Ensemble de sommets de transfert
    new_S = [] # Ensemble de sommets séurisés
    
    k_sommets = len(new_E)
    k_arcs = []
    
    for k in new_E:
        A_k = k[5]
        parcours = k[0]
        for i in range (len(A_k)):
            e = new_A[A_k[i][0]]
             
            #On ajoute la capacité au sommet
            if parcours < len(new_E):

                #C'est un noeud de départ
                if new_E[parcours][6] == None or new_E[parcours][6] > e[1]:
                    new_E[parcours][6] = e[1]
                    
            else:
    
                #C'est un noeud intermédiaire
                for j in range (len(new_T)):
                    if new_T[j][0] == parcours:
                        if new_T[j][1] == None or new_T[j][1] > e[1]:
                            new_T[j][1] = e[1]
                            
            
            if e[0] in k_arcs:
                
                #Arc connu donc rien à faire
                continue
            
            else:
                
                #On ajoute le sommet d'entrée de l'arc
                e[2] = parcours
                
                #On ajoute l'arc aux arc parcourus
                k_arcs.append(e[0])
                
                #Contrôle de l'arc suivant
                if i+1 < len(A_k):

                    #Il y a bien un arc suivant, on a affaire à un sommet intermédiaire
                    e_suiv = new_A[A_k[i+1][0]]
                    
                    #On peut donc connaître la longueur de l'arc
                    e[4] = A_k[i+1][1] - A_k[i][1]

                    if e_suiv[0] in k_arcs:

                        #Arc connu donc on connait son entrée, donc celui de sortie de l'arc actuel
                        e[3] = e_suiv[2]

                    else:

                        #Arc inconnu donc on a découvert un nouveau sommet intermédiaire
                        e[3] = k_sommets
                        new_T.append([k_sommets,None])
                        k_sommets += 1

                    #On prépare le sommet de départ pour le prochain arc
                    parcours = e[3]

                else:

                    #Pas d'arc suivant donc on a atteint un sommet de sécurité
                    e[3] = k_sommets
                    new_S.append([k_sommets,None])
                    k_sommets += 1
                    
                    #On ne peut donc pas connaître la longueur de l'arc, on la met arbitraitement à 1
                    e[4] = VALEUR_ARBITRAIRE_ARC_SORTIE
                        
                #Mise à jour de e
                new_A[e[0]] = e
                
    # TODO Bug sur le sommet 3 et arc 6 (qui a la même date au plus tôt que l'arc 4)
    # On fait le choix de l'ignorer pour le moment, on a quand même un graphe utilisable
    
    ####################### CAS DES SOMMETS SANS ARCS  #######################
            
    for k in new_E:
        A_k = k[5]
        if len(A_k) == 0:
            
            #Nouvel arc pour relier le sommet
            e = [
                len(new_A), # Identifiant de l'arc
                math.inf,   # Capacité de l'arc en personnes par unité de temps
                k[0],       # Identifiant du sommet entrant
                new_S[0][0],      # Identifiant du sommet sortant
                VALEUR_ARBITRAIRE_ARC_SORTIE,        # Longueur de l'arc
            ]
            
            A_k.append([e[0],0])
            new_A.append(e)
    
    
    ########################## TRAITEMENT DU GRAPHE ##########################
    
    X = [       # Ensemble des sommets du graphe
        new_E,  # Ensemble de sommets à évacuer
        new_T,  # Ensemble de sommets de transfert
        new_S,  # Ensemble de sommets séurisés
    ]
    
    G = [       # Notre graphe
        X,      # Ensemble des sommets du graphe
        new_A,  # Ensemble des arêtes du graphe
    ]
    
    return G

G_list = graph2list("projet/evacsim-master/data/test_10_25_2_10.evac")

print(json.dumps(G_list, indent=1))


[
 [
  [
   [
    0,
    4455,
    70,
    68,
    64,
    [
     [
      4,
      0
     ],
     [
      7,
      1
     ],
     [
      5,
      18
     ]
    ],
    132
   ],
   [
    1,
    509,
    70,
    22,
    8,
    [
     [
      0,
      0
     ],
     [
      2,
      3
     ],
     [
      7,
      6
     ],
     [
      5,
      23
     ]
    ],
    71
   ],
   [
    2,
    4584,
    71,
    128,
    65,
    [
     [
      1,
      17
     ],
     [
      5,
      30
     ]
    ],
    224
   ],
   [
    3,
    2902,
    71,
    122,
    41,
    [
     [
      4,
      8
     ],
     [
      6,
      8
     ],
     [
      7,
      9
     ],
     [
      5,
      26
     ]
    ],
    132
   ],
   [
    4,
    2808,
    71,
    84,
    40,
    [
     [
      3,
      29
     ],
     [
      1,
      35
     ],
     [
      5,
      48
     ]
    ],
    132
   ],
   [
    5,
    2744,
    70,
    22,
    40,
    [
     [
      3,
      24
     ],
     [
      1,
      30
  

Enfin, les données sont extraites pour générer le graphe qui sera donné en entrée de notre modèle de résolution.

In [4]:
def list2geosafe(G_list):
    
    nodes_state = []
    edges_state = []
    
    for eps in G_list[0][0]:
        nodes_state.append((eps[0], {'eps': True,  'r': False, 'W': eps[1], 'q': eps[6] if eps[6] is not None else math.inf, 'd': eps[3]}))
        
    for trans in G_list[0][1]:
        nodes_state.append((trans[0], {'eps': False,  'r': False, 'W': 0, 'q': trans[1] if trans[1] is not None else math.inf, 'd': 0}))
        
    for r in G_list[0][2]:
        nodes_state.append((r[0], {'eps': False,  'r': True, 'W': 0, 'q': r[1] if r[1] is not None else math.inf, 'd': 0}))
        
    for edge in G_list[1]:
        edges_state.append((edge[2], edge[3], {'l' : edge[4]}))
    
    print(nodes_state)
    print(edges_state)
    
    G = nx.DiGraph()
    G.add_nodes_from(nodes_state)
    G.add_edges_from(edges_state)
    
    return G
        
G = list2geosafe(G_list)
    

[(0, {'W': 4455, 'd': 68, 'eps': True, 'r': False, 'q': 132}), (1, {'W': 509, 'd': 22, 'eps': True, 'r': False, 'q': 71}), (2, {'W': 4584, 'd': 128, 'eps': True, 'r': False, 'q': 224}), (3, {'W': 2902, 'd': 122, 'eps': True, 'r': False, 'q': 132}), (4, {'W': 2808, 'd': 84, 'eps': True, 'r': False, 'q': 132}), (5, {'W': 2744, 'd': 22, 'eps': True, 'r': False, 'q': 132}), (6, {'W': 3982, 'd': 50, 'eps': True, 'r': False, 'q': 70}), (7, {'W': 1471, 'd': 122, 'eps': True, 'r': False, 'q': inf}), (8, {'W': 1783, 'd': 22, 'eps': True, 'r': False, 'q': 70}), (9, {'W': 358, 'd': 68, 'eps': True, 'r': False, 'q': 132}), (10, {'W': 0, 'd': 0, 'eps': False, 'r': False, 'q': 146}), (11, {'W': 0, 'd': 0, 'eps': False, 'r': False, 'q': 249}), (13, {'W': 0, 'd': 0, 'eps': False, 'r': False, 'q': 70}), (12, {'W': 0, 'd': 0, 'eps': False, 'r': True, 'q': inf})]
[(1, 13, {'l': 3}), (2, 11, {'l': 13}), (13, 10, {'l': 3}), (4, 2, {'l': 6}), (0, 10, {'l': 1}), (11, 12, {'l': 1}), (3, 10, {'l': 1}), (10, 11


## Partie 4 : 1ers Résultats  avec la contrainte cumulative

### 4.1 Présentation
Une majeure partie de notre projet a été de comprendre la modélisation a réaliser et de la réaliser de façon a respecter les contraintes imposées. Ce n'est que lors de la dernière séance que nous avons pu commencer à essayer notre modèle sur des données.  

C'est pour celà que dans cette partie nous ne présenterons qu'un simple cas de résolution, sur un petit graphe réalisé à la main. Cette étape nous parraît être la première marche de la pyramide à gravir pour résoudre un problème d'évacuation  complexe.


Le problème de la contrainte cumulative est qu'elle attend des constantes entières et non des variables dans l'interval pulse. 
Nous avons donc essayé de fixer ces valeurs :
- D'abord en fixant S, le starting date, à 0 pour tout le monde. Cependant, d'après la contrainte cumulative elle-même, il faudrait aussi fixer la durée. Or, si on fixe la durée, on doit fixer le taux h (qui nous permet d'avoir la date de fin et donc la durée). Et si on fixe h en plus de fixer s, on a fixé toutes les variables que l'on cherche à définir. Donc on n'a plus de variable à déterminer ...
```
  siuv = s[v]+l[v,u]   
  eiuv = e[v]+ l[v,u]   
  phiarc += mdl.pulse((siuv, eiuv-siuv), h[v])
            
  mdl.add( phiarc <= q[u])
```
  
avec e :
```
  e[v] == s[v] + W[v]/h[v]
```

- SOLUTION : On fixe le taux d'évacuation h. Ainsi, on a une durée fixe et on veut déterminer l'heure de départ s.

### 4.2 Modèle adapté

En utilisant la contrainte cumulative, on peut "simplifier" notre modèle qui était sans contrainte cumulative. 
D'autre part, pour essayer de résoudre un modèle simple, on fixe h, le taux d'évacuation


In [2]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *
import numpy as np

In [3]:
# CONFIG 
from config_duc import setup
setup()

In [41]:
def geosafe_model(l, q, eps, r, W, H, d,chemin):
    '''       
    Int  Matrix l: length path from i to j = G.l[i,j]. No path ij <--> G.l[i,j] = 0
    Int  Array  q: capacity of node i <--> G.q[i]
    Bool Array  eps: evacuation node 
    Bool Array  r: safe node
    Int  Array  W: initial population at node i <--> W[i]
    Int         H : Time span
    Int  Array  d: deadline to leave the node
    '''
    
    nb_node = q.shape[0] # ou len(q)
    nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
    total_population = np.sum(W)
    
    t = !eps & !r # transition node

    mdl = CpoModel(name='geosafe')
    
    # == Output ==
    # starting date
    #s = np.array([0,0,0,0,0,0])
    #s = np.array( mdl.integer_var_list(nb_node, min=0, max = H, name="s") ) # TODO : affiner max =H-w(v)/q(v)
    
    # == Fix h ==    
    # h = np.array( mdl.integer_var_list(nb_node, min=0, max = 100, name="h") ) # T0DO : affiner max =q[v]
    # evacuation rate aka. height of package
    h = np.array([5,5,5,5,5,5])
    #h = np.array([1,1,1,1,1,1])
    
    # == Interval definition for cumulative constraint ==
    itvs={}
    for v in nodes[eps]:  
        for u in chemin[v]: 
            duration = W[v]/h[v]
            itvs[v,u]=mdl.interval_var(start=[int(l[v,u]), H], size= int(duration))


    #  CUMULATIVE CONSTRAINT with a fixed h
    for v in nodes[eps]: # pour chaque resource on associe une tâche 
        # que l'on duplique ensuite sur chaque arc critique
        phiarc=step_at(0, 0)
        for u in chemin[v]: 
            phiarc += mdl.pulse(itvs[v,u], h[v])       
            mdl.add( phiarc <= q[u])
    
    
    # ==Objective ==
    mdl.add(
        #because h is fixed :
        #mdl.minimize(mdl.max(itvs[eps,] + W[eps]/h[eps] - d[eps])) 
         mdl.minimize(
             mdl.max(
                 mdl.end_of(
                        itvs[v,(chemin[v][len(chemin[v])-2])])
                 for v in nodes[eps] 
             )
         )
    )
    
    return mdl

### 4.2 Définition d'un simple graphe pour tester

In [42]:
import networkx as nx

In [43]:
G = nx.DiGraph()

global_d = 100 # TODO: delete this one, just for debug

# TODO: Maybe we should use eps = W > 0, so that do not need to use 'eps' attribute?
nodes_state = [
    (1, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (2, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (3, {'eps': True,  'r': False, 'W': 5, 'd': global_d}),
    (4, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (5, {'eps': False, 'r': False, 'W': 0, 'd': global_d}),
    (6, {'eps': False, 'r': True,  'W': 0, 'd': global_d})
]

edges_state = [
    (1, 4, {'l' : 4}),
    (2, 4, {'l' : 3}),
    (4, 5, {'l' : 7}),
    (3, 5, {'l' : 3}),
    (5, 6, {'l' : 10})
]
chemin = [
    [0,3,4,5],
    [1,3,4,5],
    [4,5]
]

G.add_nodes_from(nodes_state)
G.add_edges_from(edges_state)

In [44]:
def _node_attribute_(G, attribute='eps'):
    '''
    Author: Duc Hau :D
    Return a numpy boolean array of attribute
    '''
    values_tmp = nx.get_node_attributes(G, attribute).values()
    return np.array(list(values_tmp))

In [45]:
print(_node_attribute_(G, attribute='W'))

[5 5 5 0 0 0]


- Init Model

In [46]:
# Path length
l = nx.floyd_warshall_numpy(G, weight='l')

# Node capacities (shouldn't it be the path??)
q = np.array([20,20,20,20,20,20])

# If the node is in EPSILON (evacuating node)
eps = _node_attribute_(G, 'eps')

# If the node is in the ROOT (safe node)
r = _node_attribute_(G, 'r')

# Initial population
W = _node_attribute_(G, 'W')

# Time span
H = 200

# Deadline for evacutation
d = _node_attribute_(G, 'd')

# Solver configuration
ctx = {}

In [47]:
nb_node = q.shape[0] # ou len(q)
nodes = np.arange(nb_node) # id nodes, i.e [1,2,3,4, ...]
print("nodes ",nodes[eps])
print("l : \n ",l)


for i in nodes[eps]:
    print("i , ",i)
    print("chemin ", chemin[i])
    for j in chemin[i] :
        print(l[i,j])
        
        
chemin[0][2]

nodes  [0 1 2]
l : 
  [[ 0. inf inf  4. 11. 21.]
 [inf  0. inf  3. 10. 20.]
 [inf inf  0. inf  3. 13.]
 [inf inf inf  0.  7. 17.]
 [inf inf inf inf  0. 10.]
 [inf inf inf inf inf  0.]]
i ,  0
chemin  [0, 3, 4, 5]
0.0
4.0
11.0
21.0
i ,  1
chemin  [1, 3, 4, 5]
0.0
3.0
10.0
20.0
i ,  2
chemin  [4, 5]
3.0
13.0


4

In [48]:
mdl = geosafe_model(l, q, eps, r, W, H, d, chemin)

- Solve model

In [49]:
mdl.solve()
sol = mdl.solve(Presolve='On', Workers='Auto')
print(sol.get_solver_log())

print('\n\n Result:')
if sol.is_solution():
    sol.print_solution()
else:
    print('Problem does not have solution')
    sol.print_solution()

<ipython-input-41-db757ac7ab52>:45(stream:19:11): Warning: Cumulative expression 'stepAt': height is zero, expression is zero everywhere.
                                                             stepAt(0, 0)
<ipython-input-41-db757ac7ab52>:45(stream:26:11): Warning: Cumulative expression 'stepAt': height is zero, expression is zero everywhere.
                                                             stepAt(0, 0)
<ipython-input-41-db757ac7ab52>:45(stream:33:11): Warning: Cumulative expression 'stepAt': height is zero, expression is zero everywhere.
                                                             stepAt(0, 0)
 ! ----------------------------------------------------------------------------
 ! Minimization problem - 10 variables, 10 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 33.2 (before), 33.2 (after)
 !  . Memory usage      : 549.5 kB (before), 549.5 kB (after)
 ! Using parallel search with 4 workers.
 

**Solution pour h non contraignant** :  

On voit ici qu'en fixant h de façon non contraignante, on retrouve les données attendues que nous avons crées. C'est en quelque sorte un model checker.

```
! ----------------------------------------------------------------------------
 ! Minimization problem - 10 variables, 10 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 33.2 (before), 33.2 (after)
 !  . Memory usage      : 549.5 kB (before), 549.5 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         10                 -
 + New bound is 12
 *            12       10  0.00s        1      (gap is 0%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 12 (optimal - effective tol. is 0)
 ! Best bound             : 12
 ! Number of branches     : 12
 ! Number of fails        : 12
 ! Total memory usage     : 1.3 MB (1.2 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.00s (0.00s engine + 0.00s extraction)
 ! Search speed (br. / s) : 1200.0
 ! ----------------------------------------------------------------------------



 Result:
-------------------------------------------------------------------------------
Model constraints: 10, variables: integer: 0, interval: 10, sequence: 0
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.01 sec
-------------------------------------------------------------------------------
Objective values: (12,)
          bounds: (12,)
          gaps: (0.0,)
_ITV_41: (start=0, end=1, size=1, length=1)
_ITV_42: (start=4, end=5, size=1, length=1)
_ITV_43: (start=11, end=12, size=1, length=1)
_ITV_44: (start=21, end=22, size=1, length=1)
_ITV_45: (start=0, end=1, size=1, length=1)
_ITV_46: (start=3, end=4, size=1, length=1)
_ITV_47: (start=10, end=11, size=1, length=1)
_ITV_48: (start=20, end=21, size=1, length=1)
_ITV_49: (start=3, end=4, size=1, length=1)
_ITV_50: (start=13, end=14, size=1, length=1)
```

**Solution pour h contraignant** :  

Ici on fixe h à 1 et non plus à 5. On voit que l'on a une solution les intervals qui ont changés. De même, la borne n'est plus la même.

```
! ----------------------------------------------------------------------------
 ! Minimization problem - 10 variables, 10 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 33.2 (before), 33.2 (after)
 !  . Memory usage      : 549.5 kB (before), 549.5 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         10                 -
 + New bound is 16
 *            16       10  0.00s        1      (gap is 0%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 16 (optimal - effective tol. is 0)
 ! Best bound             : 16
 ! Number of branches     : 12
 ! Number of fails        : 12
 ! Total memory usage     : 1.3 MB (1.2 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.00s (0.00s engine + 0.00s extraction)
 ! Search speed (br. / s) : 1200.0
 ! ----------------------------------------------------------------------------



 Result:
-------------------------------------------------------------------------------
Model constraints: 10, variables: integer: 0, interval: 10, sequence: 0
Solve status: Optimal
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.01 sec
-------------------------------------------------------------------------------
Objective values: (16,)
          bounds: (16,)
          gaps: (0.0,)
_ITV_31: (start=0, end=5, size=5, length=5)
_ITV_32: (start=4, end=9, size=5, length=5)
_ITV_33: (start=11, end=16, size=5, length=5)
_ITV_34: (start=21, end=26, size=5, length=5)
_ITV_35: (start=0, end=5, size=5, length=5)
_ITV_36: (start=3, end=8, size=5, length=5)
_ITV_37: (start=10, end=15, size=5, length=5)
_ITV_38: (start=20, end=25, size=5, length=5)
_ITV_39: (start=3, end=8, size=5, length=5)
_ITV_40: (start=13, end=18, size=5, length=5)
```

## Partie 5 : Conclusion

L'équipe ADA a trouvé très interressant de travailler sur ce projet très concrêt, quoique difficile. Nous avons réussi à modéliser le problème en comprenant chaque choix éffectués par l'équipe GeoSafe. Notre modélisation avec la contrainte cumulative est maintenant prêt à être utilisé avec les données issues du générator. Il aurait été interressant de pouvoir continuer ce projet et d'arriver à trouver les résultats de GeoSafe.

L'équipe suggère pourquoi pas d'organiser une rencontre avc l'un des membre de GeoSafe si ce projet vient à se réitérer l'année prochaine. En effet, ADA aurait eu des questions à poser directement aux concepteurs.